# BTO Call/Put Screener (Multi-Horizon)

This notebook screens U.S. equities and ranks buy-to-open calls and puts across short, medium, and LEAPS horizons.

**Outputs**

- ranked candidate table by side and horizon
- chart-ready score visuals in APA-like style
- share package in `outputs/` containing CSV, Excel, and JSON manifest


In [ ]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

from notebook_pipeline import (
    setup_report_style,
    parse_env_list,
    display_table,
    screen_universe,
    fetch_underlying_metrics,
    fetch_option_candidates,
    score_option_candidates,
    top_by_bucket,
    build_correlation_matrix,
)

setup_report_style()

FIGURE_COUNTER = 0


def show_figure(fig, title):
    global FIGURE_COUNTER
    FIGURE_COUNTER += 1
    if not str(title).lower().startswith("figure"):
        title = f"Figure {FIGURE_COUNTER}. {title}"
    fig.update_layout(title=title)
    fig.show()


In [ ]:
USE_SCREEN = os.getenv("USE_SCREEN", "1") == "1"
QUICK_RUN = os.getenv("QUICK_RUN", "0") == "1"
TICKER_OVERRIDE = parse_env_list("TICKER_OVERRIDE")

MAX_TICKERS = int(os.getenv("MAX_TICKERS", "30"))
RATE_LIMIT_SLEEP = float(os.getenv("RATE_LIMIT_SLEEP", "0.20"))


if QUICK_RUN:
    MAX_TICKERS = min(MAX_TICKERS, 8)
    RATE_LIMIT_SLEEP = min(RATE_LIMIT_SLEEP, 0.10)

HORIZONS = {
    "short": {"min_dte": 7, "max_dte": 35, "target_dte": 21},
    "medium": {"min_dte": 36, "max_dte": 140, "target_dte": 75},
    "leaps": {"min_dte": 250, "max_dte": 760, "target_dte": 420},
}

CALL_MONEYNESS = {
    "short": (0.95, 1.08),
    "medium": (0.90, 1.12),
    "leaps": (0.75, 1.08),
}
PUT_MONEYNESS = {
    "short": (0.92, 1.08),
    "medium": (0.88, 1.15),
    "leaps": (0.80, 1.18),
}

config_df = pd.DataFrame(
    {
        "Parameter": [
            "USE_SCREEN",
            "QUICK_RUN",
            "MAX_TICKERS",
            "RATE_LIMIT_SLEEP",
            "TICKER_OVERRIDE",
        ],
        "Value": [
            USE_SCREEN,
            QUICK_RUN,
            MAX_TICKERS,
            RATE_LIMIT_SLEEP,
            ", ".join(TICKER_OVERRIDE) if TICKER_OVERRIDE else "<none>",
        ],
    }
)
display_table(config_df, caption="Run Configuration")


,Parameter,Value
0,USE_SCREEN,True
1,QUICK_RUN,False
2,MAX_TICKERS,30
3,RATE_LIMIT_SLEEP,0.200000
4,OUTPUT_DIR,outputs
5,TICKER_OVERRIDE,


In [19]:
tickers = screen_universe(
    use_screen=USE_SCREEN,
    ticker_override=TICKER_OVERRIDE,
    max_tickers=MAX_TICKERS,
    size=max(60, MAX_TICKERS * 3),
)

display(Markdown(f"**Tickers selected:** {len(tickers)}"))
display(pd.DataFrame({"Ticker": tickers[:30]}))

underlying_df = fetch_underlying_metrics(
    tickers,
    history_period="1y",
    rate_limit_sleep=RATE_LIMIT_SLEEP,
)
if underlying_df.empty:
    raise RuntimeError("No underlying data available. Check network/data source.")

display_table(
    underlying_df[
        [
            "ticker",
            "sector",
            "spot",
            "ret_1m",
            "ret_3m",
            "hv_30",
            "rsi_14",
            "avg_volume_3m",
        ]
    ].sort_values("ret_3m", ascending=False),
    caption="Underlying Metrics",
    format_dict={
        "spot": "${:,.2f}",
        "ret_1m": "{:.1%}",
        "ret_3m": "{:.1%}",
        "hv_30": "{:.1%}",
        "rsi_14": "{:.1f}",
        "avg_volume_3m": "{:,.0f}",
    },
)

candidate_frames = []
for _, u in underlying_df.iterrows():
    spot = float(u["spot"])
    ticker = u["ticker"]

    calls = fetch_option_candidates(
        ticker,
        side="call",
        spot=spot,
        horizons=HORIZONS,
        moneyness_bounds=CALL_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )
    puts = fetch_option_candidates(
        ticker,
        side="put",
        spot=spot,
        horizons=HORIZONS,
        moneyness_bounds=PUT_MONEYNESS,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )

    if not calls.empty:
        candidate_frames.append(calls)
    if not puts.empty:
        candidate_frames.append(puts)

candidates_df = (
    pd.concat(candidate_frames, ignore_index=True)
    if candidate_frames
    else pd.DataFrame()
)
if candidates_df.empty:
    raise RuntimeError(
        "No option candidates found. Relax filters or try different tickers."
    )

scored_df = score_option_candidates(candidates_df, underlying_df)
leaderboard_df = top_by_bucket(
    scored_df,
    bucket_cols=["side", "horizon"],
    top_n=int(os.getenv("TOP_PER_BUCKET", "8")),
)

best_tsh = (
    scored_df.sort_values("master_score", ascending=False)
    .groupby(["ticker", "side", "horizon"], as_index=False)
    .first()
)

display_table(
    leaderboard_df[
        [
            "side",
            "horizon",
            "ticker",
            "contract_symbol",
            "dte",
            "strike",
            "mid",
            "iv",
            "expected_return",
            "master_score",
        ]
    ],
    caption="Leaderboard: Top Contracts per Side/Horizon",
    format_dict={
        "strike": "${:,.2f}",
        "mid": "${:,.2f}",
        "iv": "{:.1%}",
        "expected_return": "{:.1%}",
        "master_score": "{:.1f}",
    },
)


**Tickers selected:** 30

,Ticker
0,AMZN
1,INTC
2,IREN
3,SOFI
4,KVUE
5,TSLA
6,PLTR
7,MSTR
8,GOOGL
9,AAL


,ticker,sector,spot,ret_1m,ret_3m,hv_30,rsi_14,avg_volume_3m
26,MU,Technology,$394.69,16.2%,66.3%,71.5%,57.5,"31,883,600"
22,CFLT,Technology,$30.57,1.3%,35.6%,4.4%,52.6,"14,754,485"
1,INTC,Technology,$50.59,18.7%,31.8%,91.4%,55.3,"102,093,495"
28,VZ,Communication Services,$46.31,17.4%,18.7%,36.4%,88.6,"29,602,578"
15,PFE,Healthcare,$27.22,9.5%,14.4%,22.8%,66.8,"57,102,609"
8,GOOGL,Communication Services,$322.86,0.3%,13.6%,19.1%,43.7,"37,314,380"
9,AAL,Industrials,$15.24,-4.7%,13.6%,44.2%,48.8,"55,018,578"
4,KVUE,Consumer Defensive,$18.13,8.3%,13.1%,18.1%,68.0,"42,372,155"
24,T,Communication Services,$27.13,14.5%,11.8%,26.1%,84.6,"42,458,695"
21,BAC,Financial Services,$56.53,1.6%,8.3%,23.4%,71.1,"37,394,500"


,side,horizon,ticker,contract_symbol,dte,strike,mid,iv,expected_return,master_score
0,put,short,PYPL,PYPL260306P00040000,26,$40.00,$1.50,39.9%,1134.2%,90.7
1,put,medium,PYPL,PYPL260417P00040000,68,$40.00,$2.42,39.1%,728.9%,90.5
2,put,leaps,PYPL,PYPL261218P00040000,313,$40.00,$5.67,41.0%,338.2%,90.4
3,put,leaps,PYPL,PYPL270115P00040000,341,$40.00,$5.82,40.6%,333.6%,90.2
4,call,leaps,F,F270115C00012000,341,$12.00,$0.81,0.0%,380.0%,90.1
5,put,leaps,PYPL,PYPL270115P00035000,341,$35.00,$3.55,41.7%,470.7%,89.9
6,put,medium,AMD,AMD260417P00230000,68,$230.00,$31.83,50.5%,165.3%,89.6
7,put,medium,PYPL,PYPL260417P00037500,68,$37.50,$1.42,40.6%,1139.6%,89.4
8,put,medium,MSFT,MSFT260417P00450000,68,$450.00,$51.95,26.4%,200.9%,89.3
9,put,leaps,PYPL,PYPL270115P00037500,341,$37.50,$4.78,42.9%,376.6%,89.2


In [23]:
plot_df = leaderboard_df.copy()
if not plot_df.empty:
    fig_bar = px.bar(
        plot_df.sort_values("master_score", ascending=True),
        x="master_score",
        y="ticker",
        color="side",
        facet_col="horizon",
        orientation="h",
        hover_data=["contract_symbol", "dte", "expected_return", "iv"],
        height=600,
        color_discrete_sequence=["#1F3A5F", "#B0533C"],
    )
    show_figure(fig_bar, "Top Contract Scores by Horizon and Side")

    fig_scatter = px.scatter(
        plot_df,
        x="iv",
        y="expected_return",
        color="master_score",
        symbol="side",
        facet_col="horizon",
        hover_data=["ticker", "contract_symbol", "dte", "mid"],
        color_continuous_scale="Blues",
        height=600,
    )
    fig_scatter.update_yaxes(tickformat=".0%")
    fig_scatter.update_xaxes(tickformat=".0%")
    show_figure(fig_scatter, "IV Level vs Expected Return")


In [24]:
corr_matrix = build_correlation_matrix(
    leaderboard_df["ticker"].unique().tolist(), period="6mo"
)
if corr_matrix.empty:
    display(
        Markdown(
            "Correlation matrix unavailable (insufficient overlapping return history)."
        )
    )
else:
    fig_corr = go.Figure(
        data=go.Heatmap(
            z=corr_matrix.values,
            x=corr_matrix.columns,
            y=corr_matrix.index,
            colorscale="RdBu_r",
            zmid=0,
            zmin=-1,
            zmax=1,
        )
    )
    show_figure(fig_corr, "Underlying Correlation Matrix")


In [ ]:
display(Markdown("## Run Summary"))

summary_lines = [
    f"- **Tickers screened:** {len(tickers)}",
    f"- **Total contracts scored:** {len(scored_df):,}",
    f"- **Leaderboard entries:** {len(leaderboard_df)}",
    f"- **Unique tickers on leaderboard:** {leaderboard_df['ticker'].nunique()}",
]
if not corr_matrix.empty:
    avg_corr = corr_matrix.values[np.triu_indices_from(corr_matrix.values, k=1)].mean()
    summary_lines.append(f"- **Average portfolio correlation:** {avg_corr:.2f}")

display(Markdown("\n".join(summary_lines)))

if not best_tsh.empty:
    display(Markdown("### Top Pick Per Ticker, Side, and Horizon"))
    display_table(
        best_tsh[
            ["ticker", "side", "horizon", "strike", "mid", "dte", "master_score"]
        ].head(20),
        caption="Best Contract by Ticker / Side / Horizon",
        format_dict={
            "strike": "${:,.2f}",
            "mid": "${:,.2f}",
            "master_score": "{:.1f}",
        },
    )
